In [ ]:
obs_file ="/neurospin/lrmn/people/Zaineb_Amor/DATAOctober2021/12192021_InVivo_rd210175/meas_MID190_3D_Snorekling_1_2mmiso_1_638s_176rep_Clock_FID5960.dat"
#traj_file = "/neurospin/lrmn/people/Zaineb_Amor/DATASeptembre2021/09092021_Traj_Snork_Utilisée_InVivo/dim3_i_RadialIO_P0.75_N192x192x128_FOV0.192x0.192x0.128_Nc8_Ns2688_c25_d2__D9M9Y2021T1017_reproject.bin"
traj_file = "/neurospin/lrmn/people/Zaineb_Amor/DATAOctober2021/trajs_0310/dim3_i_RadialIO_P0.75_N160x160x104_FOV0.192x0.192x0.1248_Nc13_Ns2176_c25_d2__D4M10Y2021T1016_reproject.bin" 
smaps_file ="/neurospin/lrmn/people/Zaineb_Amor/DATAOctober2021/12192021_InVivo_rd210175/Smaps/Smaps1_2mm.npy"

In [ ]:
import pysap
from mri.operators import Stacked3DNFFT, NonCartesianFFT, WaveletN, FFT, OWL
from mri.operators.utils import normalize_frequency_locations
from mri.reconstructors import SelfCalibrationReconstructor,CalibrationlessReconstructor
from mri.operators.fourier.utils import estimate_density_compensation
from mri.reconstructors.utils.extract_sensitivity_maps import get_Smaps
from modopt.math.metrics import psnr, nrmse, ssim
import pydicom
import os
import numpy as np
from twixreader import Twix
import pickle as pkl
import argparse
import datetime
from datetime import datetime
from modopt.opt.linear import Identity
from modopt.opt.proximity import SparseThreshold, GroupLASSO
from sparkling.utils.gradient import get_kspace_loc_from_gradfile
from sparkling.utils.shots import convert_NCxNSxD_to_NCNSxD, convert_NCNSxD_to_NCxNSxD
import numpy as np
import re
import sys
import mapvbvd

In [ ]:
def get_raw_data(filename):
    # Function that reads a SIEMENS .dat file and returns a k space data
    data = Twix(filename)[-1]['ima'].raw()
    data = np.swapaxes(data, 1, 2)
    data = np.reshape(data, (data.shape[0] * data.shape[1], data.shape[2]))
    return data.T

In [ ]:
def get_samples(filename, dwell_t, num_adc_samples, kmax):
    sample_locations = convert_NCxNSxD_to_NCNSxD(get_kspace_loc_from_gradfile(filename, dwell_t, num_adc_samples)[0])
    sample_locations = normalize_frequency_locations(sample_locations, Kmax=kmax)
    return sample_locations

In [ ]:
def add_phase_kspace(kspace_data, kspace_loc, shifts={}):
    if shifts == {}:
        shifts = (0,) * kspace_loc.shape[1]
    if len(shifts) != kspace_loc.shape[1]:
        raise ValueError("Dimension mismatch between shift and kspace locations! "
                         "Ensure that shifts are right")
    phi = np.zeros_like(kspace_data)
    for i in range(kspace_loc.shape[1]):
        phi += kspace_loc[:, i] * shifts[i]
    phase = np.exp(-2 * np.pi * 1j * phi)
    return kspace_data * phase

In [ ]:
#set reconstruction variables 
M = [160, 160,104]
FOV = [0.192, 0.192, 0.1248]
Ns = 2176
OS = 5
#get the data samples and the kspace samples
kspace_loc = []
kspace_data = []
kspace_data = get_raw_data(obs_file)
#kspace_data_twixObj = mapvbvd.mapVBVD(obs_file)
#kspace_data_twixObj.search_header_for_keys('adFree')
kspace_loc= get_samples(traj_file, 0.01/OS, Ns*OS,kmax=(M[0]/(2*FOV[0]), M[1]/(2*FOV[1]), M[2]/(2*FOV[2])))

In [ ]:
shifts_mm = (-25.4*0.001, 0.0, -13.1*0.001)
shifts_slice = ((shifts_mm[0]*M[0])/FOV[0],(shifts_mm[1]*M[1])/FOV[1],(shifts_mm[2]*M[2])/FOV[2])
kspace_data.shape, kspace_loc.shape

In [ ]:
kspace_data1 =  add_phase_kspace(kspace_data[:,10*424320:11*424320], kspace_loc, shifts=shifts_slice)
del(kspace_data)

In [ ]:
smaps = np.load(smaps_file,allow_pickle=True, fix_imports=True)
mu = 1e-8
niter = 15

In [ ]:
density_comp = estimate_density_compensation(kspace_loc, M)

fourier_op = NonCartesianFFT(samples=kspace_loc,
                                     shape=M,
                                     implementation='gpuNUFFT',
                                     density_comp=density_comp,
                                     n_coils=32 )
regularizer_op = SparseThreshold(Identity(), mu, thresh_type="soft")

linear_op = WaveletN(wavelet_name='sym8',
                    nb_scale=3,
                    dim=3,
                    padding='periodization')

reconstructor = SelfCalibrationReconstructor(
                fourier_op=fourier_op,
                linear_op=linear_op,
                regularizer_op=regularizer_op,
                gradient_formulation='synthesis',
                Smaps=smaps,
                n_jobs=1,
                verbose=21
            )

reconst_data1, costs, metrics = reconstructor.reconstruct(kspace_data=kspace_data1,
                                                                 optimization_alg='pogm', num_iterations=niter,
                                                                 recompute_smaps=False,
                                                                 #cost_op_kwargs={'cost_interval': 1}
                                                         )

In [ ]:
import matplotlib.pyplot as plt 
#plt.imshow(abs(np.roll(reconst_data1[:,:,64], 30)), cmap='gray')
plt.imshow(abs(reconst_data1[:,:,64]), cmap='gray')
np.save("/neurospin/lrmn/people/Zaineb_Amor/DATAOctober2021/12192021_InVivo_rd210175/10_8_1_2mm", reconst_data1)

In [ ]:
_3 = np.load("/neurospin/lrmn/people/Zaineb_Amor/DATAOctober2021/exvivo_data_0410/3x13.npy")
_13 = np.load("/neurospin/lrmn/people/Zaineb_Amor/DATAOctober2021/exvivo_data_0410/13x3.npy")
#_7 = np.load("")

In [ ]:
import nibabel as nib
vol_epi = nib.load("/neurospin/lrmn/people/Zaineb_Amor/DATAOctober2021/exvivo_data_0410/EPI0410MAG_EPI3D_1_2mmiso_tr1_638s_1rep_PA_20211004194344_4.nii.gz")
vol_epi = vol_epi.get_fdata()
vol_epi.shape

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
plt.figure(figsize=(20,20))
plt.subplot(1,3,1)
plt.imshow(abs(_3[:,:,45]), cmap="gray")
plt.subplot(1,3,2)
plt.imshow(abs(_13[:,:,45]), cmap="gray")
plt.subplot(1,3,3)
plt.imshow(abs(vol_epi[:,73,:]), cmap="gray")

In [ ]:
plt.figure(figsize=(20,20))
plt.subplot(1,3,1)
plt.imshow(abs(_3[:,40,:]), cmap="gray")
plt.subplot(1,3,2)
plt.imshow(abs(_13[:,40,:]), cmap="gray")
plt.subplot(1,3,3)
plt.imshow(abs(vol_epi[:,:,50]), cmap="gray")